In [1]:
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse.linalg as splin

# data loading

## 20M

In [2]:
#20 millions

#links = pd.read_csv('ml-20m/links.csv')
#df_ratings = pd.read_csv('ml-20m/ratings.csv')
#df_ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
#df_ratings.head()


#1 million

df_ratings = pd.read_csv('ml-1m/ratings.dat', sep="::", header=None)
#df_ratings.columns = "UserID::MovieID::Rating::Timestamp".split("::") 
df_ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df_ratings.head()

<ipython-input-2-9957857c0649>:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_ratings = pd.read_csv('ml-1m/ratings.dat', sep="::", header=None)


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
all_cells = df_ratings.movie_id.nunique() * df_ratings.user_id.nunique()
sparcity_rate = len(df_ratings) / all_cells
sparcity_rate

0.044683625622312845

In [4]:
df_ratings['user_id'] =  df_ratings['user_id'].astype('str')
df_ratings['movie_id'] =  df_ratings['movie_id'].astype('str')

dense_rating = df_ratings.pivot(index='user_id', columns='movie_id', values='rating')
dense_rating = dense_rating.fillna(0)

In [5]:
dense_rating

movie_id,1,10,100,1000,1002,1003,1004,1005,1006,1007,...,99,990,991,992,993,994,996,997,998,999
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
997,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## SVD

In [6]:
def svd_collaborative_filr(M, r = 6):
    U, S, V = splin.svds(M.values, k = r) # Update this line, it is just example

    S_diag = np.diag(S) 
    weights = np.array([S[i] * V[i] for i in range(len(S))])
    
    return weights

In [7]:
weights_from_svd = svd_collaborative_filr(dense_rating)

In [8]:
weights_from_svd.shape

(6, 3706)